# TME1

In [41]:
import numpy as np

## Recup data

In [86]:
file = open("CTR.txt","r")
txt = file.read()
file.close()

lines = txt.split("\n")
lines = lines[:len(lines)-1]

data = []
for line in lines:
    tmp = line.split(":")
    dic = {"id":tmp[0], "article":tmp[1].split(";"), "annonceurs":tmp[2].split(";")}
    data.append(dic)

{'id': '0',
 'article': ['0.74837091377052',
  '0.8352077827766918',
  '0.07669895743095312',
  '0.17243898920810596',
  '0.14344585283799294'],
 'annonceurs': ['0.10341905704918021',
  '0.19069778281037159',
  '0.0',
  '0.10240139196802925',
  '0.0363124254124334',
  '0.07456195450906297',
  '0.2347024151796051',
  '0.0',
  '0.0',
  '0.07857372422425657']}

In [ ]:
class Bandit():
    

In [45]:
def random(t, gains, nbs, gainOpti, statsTricheur, gainMoyen):
    """
    
    :type t: int
    :type gains: [int]*10 gain cumulés à l'instant t
    :type nbs: [int]*10 nombre de fois où un bras a été joué à l'instant t
    :type gainOpti: [int]*10 gain au temps t
    :type statsTricheur: [int]*2 gain cumulés des strategies tricheur
    :type gainMoyen: [int]*10 gain moyen en prenant toujours le même bras
    
    """
    return np.random.randInt(len(stats))

In [46]:
def staticBest(t, gains, nbs, gainOpti, statsTricheur, gainMoyen):
    """
    
    :type t: int
    :type gains: [int]*10 gain cumulés à l'instant t
    :type nbs: [int]*10 nombre de fois où un bras a été joué à l'instant t
    :type gainOpti: [int]*10 gain au temps t
    :type statsTricheur: [int]*2 gain cumulés des strategies tricheur
    :type gainMoyen: [int]*10 gain moyen en prenant toujours le même bras
    
    """
    return np.argmax(gainMoyen)

In [ ]:
def optimale(t, gains, nbs, gainOpti, statsTricheur, gainMoyen):
    """
    
    :type t: int
    :type gains: [int]*10 gain cumulés à l'instant t
    :type nbs: [int]*10 nombre de fois où un bras a été joué à l'instant t
    :type gainOpti: [int]*10 gain au temps t
    :type statsTricheur: [int]*2 gain cumulés des strategies tricheur
    :type gainMoyen: [int]*10 gain moyen en prenant toujours le même bras
    
    """
    return np.argmax(gainOpti)

In [44]:
def ucb(t, gains, nbs, gainOpti, statsTricheur, gainMoyen):
    """
    
    :type t: int
    :type gains: [int]*10 gain cumulés à l'instant t
    :type nbs: [int]*10 nombre de fois où un bras a été joué à l'instant t
    :type gainOpti: [int]*10 vrai gains au temps t
    :type statsTricheur: [int]*2 gain cumulés des strategies tricheur     ordre =  StaticOpti; Opti
    :type gainMoyen: [int]*10 gain moyen en prenant toujours le même bras  
    
    """
    return np.argmax((gains/nbs)+np.sqrt(2*np.log(t)/nbs))

1

In [60]:
choiceFunc = random

In [61]:
nbBras = 10

In [91]:
def getGainMoyen(data):
    gainCum = np.array([0.0]*nbBras, dtype=np.float64)
    for d in data:
        gainCum = gainCum + np.array(d['annonceurs'], dtype=np.float64)
    return gainCum/len(data)

In [94]:

t = 0
gains = np.array([0.0]*nbBras)
nbs = np.array([0.0]*nbBras)
statsTricheur = [0.0, 0.0]
gainMoyen = getGainMoyen(data)



for d in data:
    gainOpti = d['annonceurs']
    if(t<nbBras):
        choice = t
    else:
        choice = choiceFunc(t, gains, nbs, gainOpti, statsTricheur, gainMoyen)
    
    t += 1
    gains[choice] += gainOpti[choice]
    nbs[choice] += 1
    statsTricheur[0] += gainOpti[staticBest(t, gains, nbs, gainOpti, statsTricheur, gainMoyen)]
    statsTricheur[1] += gainOpti[optimale(t, gains, nbs, gainOpti, statsTricheur, gainMoyen)]

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [92]:
getGainMoyen(data)

array([0.01645323, 0.07724607, 0.00735922, 0.26792701, 0.03255289,
       0.00230823, 0.09774602, 0.0355437 , 0.00616893, 0.24596128])

In [55]:
t = np.array([1,2,3])
t += np.array([10,20,30])

In [56]:
t

array([11, 22, 33])

In [80]:
len(data)

0

In [81]:
data[0]

IndexError: list index out of range